## Initialize credentials to access migdar data

Start your notebook like this to  set the required env vars:

```
env MIGDAR_USERNAME="" MIGDAR_PASSWORD="" jupyter lab
```

## Download the aggregated search results

This data is not updated, just need to download it once

In [5]:
%%bash

# comment to download all rows (It's 470MB)
LIMIT_ROWS="3000"

./download_search_results_unique_records.sh "${LIMIT_ROWS}"

3000


In [6]:
from dataflows import Flow, load, printer
Flow(load('data/search_results/unique_records.csv'),
     printer(tablefmt='html', fields=['title', 'pubyear', 'migdar_id'], num_rows=1)).process()[1]

#,title (string),pubyear (string),migdar_id (integer)
1,הוראות בלתי שגרתיות לתפירת צואר הרחם,1981,0
2,שינויים בזרימת הדם בשורר הרחם בעקבות הזרקת פרוסטגלנדין ALPHA F2 להפסקת הריון בטרימסטר שני,1984,1
...,,,
2999,Fleeing the house of horrors women who have left abusive partners /,2002,2998


{}

## Load the data from GDrive

In [10]:
!{'dpp run ./load_publications_from_gdrive'}


./load_publications_from_gdrive: WAITING FOR OUTPUT

./load_publications_from_gdrive: RUNNING, processed 100 rows

./load_publications_from_gdrive: RUNNING, processed 200 rows

./load_publications_from_gdrive: RUNNING, processed 300 rows

./load_publications_from_gdrive: RUNNING, processed 400 rows

./load_publications_from_gdrive: RUNNING, processed 500 rows

./load_publications_from_gdrive: RUNNING, processed 600 rows

./load_publications_from_gdrive: RUNNING, processed 700 rows

./load_publications_from_gdrive: RUNNING, processed 800 rows

./load_publications_from_gdrive: RUNNING, processed 900 rows

./load_publications_from_gdrive: RUNNING, processed 1000 rows

./load_publications_from_gdrive: RUNNING, processed 1100 rows

./load_publications_from_gdrive: RUNNING, processed 1200 rows

./load_publications_from_gdrive: RUNNING, processed 1300 rows

./load_publications_from_gdrive: RUNNING, processed 1400 rows

./load_publications_from_gdrive: RUNNING, processed 1500 rows

./load_pub

## Join with the source data

The GDrive data contains the user-entered data, this joins it with the source data and full marc data

In [14]:
!{'dpp run ./join_publications_unique_records,./join_publications_search_app_records'}


./join_publications_unique_records: WAITING FOR OUTPUT

./join_publications_unique_records: RUNNING, processed 100 rows

./join_publications_unique_records: RUNNING, processed 200 rows

./join_publications_unique_records: RUNNING, processed 300 rows

./join_publications_unique_records: RUNNING, processed 389 rows

./join_publications_unique_records: SUCCESS, processed 389 rows

./join_publications_unique_records: SUCCESS, processed 389 rows
./join_publications_search_app_records: WAITING FOR OUTPUT

./join_publications_unique_records: SUCCESS, processed 389 rows
./join_publications_search_app_records: WAITING FOR OUTPUT

./join_publications_unique_records: SUCCESS, processed 389 rows
./join_publications_search_app_records: SUCCESS, processed 0 rows
INFO    :RESULTS:
INFO    :SUCCESS: ./join_publications_unique_records {'bytes': 2754976, 'count_of_rows': 389, 'dataset_name': '_', 'hash': '854672bc0d50b2cf249e3f5b5558445b'}
INFO    :SUCCESS: ./join_publications_search_app_records {'byte

In [15]:
from dataflows import Flow, load, printer

Flow(
    load('data/unique_records_full/datapackage.json'),
    load('data/app_records_full/datapackage.json'),
    printer(tablefmt='html', num_rows=1, fields=['migdar_id', 'title', 'pubyear', 'publisher', 'author', 'language_code'])
).process()[1]

#,title (string),pubyear (string),publisher (string),author (string),language_code (string),migdar_id (string)
1,הפסקות הריון על פי החוק : 2003-1990 /,"תשס""ה 2005.","תחום מידע - שירותי מידע ומחשוב, משרד הבריאות,","חקלאי, ציונה",heb,178
2,הפסקות הריון על פי החוק : 2004-1990 /,"תשס""ו 2006.","תחום מידע - שירותי מידע ומחשוב, משרד הבריאות,","חקלאי, ציונה",heb,186
...,,,,,,
389,The enigma woman the death sentence of Nellie May Madison /,c2007.,"University of Nebraska Press,","Cairns, Kathleen A., 1946-",,2993


{}

## Prepare data for ElasticSearch

Start the docker-compose stack for the required infra.

In [2]:
!{'docker-compose up -d'}

latest: Pulling from library/adminer
Digest: sha256:279918485345053f5a84561ca4658786d7f8c91b35ef10ea2aa2bf2ef4699fa9
Status: Downloaded newer image for adminer:latest


Creating network "migdardatapipelines_default" with the default driver
Creating volume "migdardatapipelines_db" with default driver
Pulling adminer (adminer:latest)...
Creating migdardatapipelines_db_1 ... 
Creating migdardatapipelines_db_1
Creating migdardatapipelines_adminer_1 ... 
Creating migdardatapipelines_adminer_1


In [65]:
%%bash
while ! [ "$(docker-compose ps | grep ' Up ' | wc -l)" == "3" ]; do
    echo .
    sleep 2
done &&\
docker-compose ps

               Name                              Command               State                 Ports              
----------------------------------------------------------------------------------------------------------------
migdardatapipelines_adminer_1         entrypoint.sh docker-php-e ...   Up      0.0.0.0:18080->8080/tcp          
migdardatapipelines_db_1              docker-entrypoint.sh postgres    Up      0.0.0.0:15432->5432/tcp          
migdardatapipelines_elasticsearch_1   /docker-entrypoint.sh elas ...   Up      0.0.0.0:19200->9200/tcp, 9300/tcp


Bump the revision

In [6]:
import os

with open('datapackage_pipelines_migdar/flows/PUBLICATIONS_REVISION.txt', 'r') as f:
    cur_revision = f.read().strip()
new_revision = int(cur_revision) + 1
with open('datapackage_pipelines_migdar/flows/PUBLICATIONS_REVISION.txt', 'w') as f:
    f.write(str(new_revision))
    
print(f'PUBLICATIONS_REVISION = {new_revision}')

PUBLICATIONS_REVISION = 12


In [9]:
!{'dpp run ./prepare_data_for_es'}


./prepare_data_for_es: WAITING FOR OUTPUT

./prepare_data_for_es: RUNNING, processed 100 rows

./prepare_data_for_es: RUNNING, processed 200 rows

./prepare_data_for_es: RUNNING, processed 300 rows

./prepare_data_for_es: RUNNING, processed 389 rows

./prepare_data_for_es: SUCCESS, processed 389 rows
INFO    :RESULTS:
INFO    :SUCCESS: ./prepare_data_for_es {'bytes': 2937751, 'count_of_rows': 389, 'dataset_name': '_', 'hash': 'dbaa6f872c7d5698c0ec532f44899b52'}


In [48]:
from dataflows import Flow, load, printer, update_resource
from datapackage_pipelines_migdar.flows.common import DATAFLOWS_DB_ENGINE
from datapackage_pipelines_migdar.flows.constants import PUBLICATIONS_DB_TABLE
from tabulator import Stream
from tabulate import tabulate
from IPython.display import HTML
import yaml

preview_fields = ['title', 'migdar_id', 'gd_Item Type', 'gd_Life Domains', '__revision', '__is_new', '__is_stale', '__hash']

def filter_db_row(row):
    return {k: v for k, v in row.items()
            if k in preview_fields}
    

with Stream(DATAFLOWS_DB_ENGINE, table=PUBLICATIONS_DB_TABLE, headers=1) as stream:
    display(HTML(f'<h3>{PUBLICATIONS_DB_TABLE}</h3>'))
    table = ''
    table += '<table>'
    for i, row in enumerate(stream.iter(keyed=True)):
        if i < 2:
            row = filter_db_row(row)
            table += f'<tr><td>#{i}</td><td style="text-align:left;">{row}</td></tr>'
    table += ('<tr><td>...</td><td></td></tr>')
    if i > 2:
        row = filter_db_row(row)
        table += f'<tr><td>#{i}</td><td style="text-align:left;">{row}</td></tr>'
    table += '</table>'
    display(HTML(table))

Flow(
    load('data/publications_for_es/datapackage.json'),
    printer(tablefmt='html', num_rows=1, resources=['unique_records', 'search_app_records'], fields=preview_fields)
).process()[1]

#0,"{'title': 'הפסקות הריון על פי החוק : 2003-1990 /', 'migdar_id': '178', 'gd_Item Type': 'Datasheet', 'gd_Life Domains': 'Health', '__revision': 12, '__is_new': False, '__is_stale': False, '__hash': '58791090cd35d028445e2371c99c10a0'}"
#1,"{'title': 'סוגיות במשפט העברי /', 'migdar_id': '191', 'gd_Item Type': 'Book', 'gd_Life Domains': 'Religion', '__revision': 12, '__is_new': False, '__is_stale': False, '__hash': 'd6694fe3ee9294c638de6b2e152fe068'}"
...,
#388,"{'title': 'The enigma woman the death sentence of Nellie May Madison /', 'migdar_id': '2993', 'gd_Item Type': 'Book', 'gd_Life Domains': 'Gender Violence', '__revision': 12, '__is_new': False, '__is_stale': False, '__hash': 'e17f4b3a7bc310f82bcb742299ec65e9'}"


#,title (string),migdar_id (string),gd_Item Type (string),gd_Life Domains (string),__revision (integer),__is_new (boolean),__is_stale (boolean),__hash (string)
1,הפסקות הריון על פי החוק : 2003-1990 /,178,Datasheet,Health,12,False,False,58791090cd35d028445e2371c99c10a0
2,הפסקות הריון על פי החוק : 2004-1990 /,186,Datasheet,Health,12,False,False,cfe6517c8281e731fc3570bf20aaf971
...,,,,,,,,
389,The enigma woman the death sentence of Nellie May Madison /,2993,Book,Gender Violence,12,False,False,e17f4b3a7bc310f82bcb742299ec65e9


{}

## dump to es

In [50]:
!{'dpp run ./dump_publications_to_es'}


./dump_publications_to_es: WAITING FOR OUTPUT

./dump_publications_to_es: RUNNING, processed 100 rows

./dump_publications_to_es: RUNNING, processed 200 rows

./dump_publications_to_es: RUNNING, processed 300 rows

./dump_publications_to_es: RUNNING, processed 389 rows

./dump_publications_to_es: SUCCESS, processed 389 rows
INFO    :RESULTS:
INFO    :SUCCESS: ./dump_publications_to_es {}


In [62]:
import yaml, json

res = !{'curl -s http://localhost:19200/migdar/_search'}
print(yaml.dump(json.loads(res[0]), default_flow_style=False, allow_unicode=True))

_shards:
  failed: 0
  successful: 5
  total: 5
hits:
  hits:
  - _id: '190'
    _index: migdar_20181024225043071808_2d3d7d41
    _score: 1.0
    _source:
      __created_at: '2018-10-24T21:21:57'
      __date_range_from: '2005-01-01'
      __date_range_months:
      - 2005-01
      - 2005-02
      - 2005-03
      - 2005-04
      - 2005-05
      - 2005-06
      - 2005-07
      - 2005-08
      - 2005-09
      - 2005-10
      - 2005-11
      - 2005-12
      __date_range_to: '2005-12-31'
      __hash: c6d98dfdccf39a36fc886188be9cd8dc
      __is_new: false
      __is_stale: false
      __last_modified_at: null
      __last_updated_at: '2018-10-25T11:55:31'
      __next_update_days: 2
      __revision: 12
      __staleness: 150000
      author: גולן, אילנה
      custom_metadata: 2006 A 3247=2
      doc_id: publications/190
      first_ccl_query: '"Abortion" heb'
      gd_Item Type: Academic Research
      gd_Life Domains: Healt
      gd_Resource Type: Academic Institutions
      gd_author: 